In [68]:
# install keras==2.3.1
# !pip install tensorflow==2.1.0
# !pip install keras_applications==1.0.8
# !pip install image-classifiers==1.0.0
# !pip install efficientnet==1.0.0

In [69]:
%matplotlib inline
import torch
import torch.optim as optim
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import sys
from torch.utils.data import DataLoader


#Our project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname("__file__"), 
                                            os.pardir))
sys.path.append(PROJECT_ROOT)

from util.constants import DATA_PATHS
from util.dataset import OcelotDatasetLoader

import tensorflow as tf
import segmentation_models as sm
# BACKBONE = 'resnet34'
# preprocess_input = sm.get_preprocessing(BACKBONE)

my_device = torch.device(device = 'cuda' if torch.cuda.is_available() else 'cpu')
pin_memory = True if my_device == 'cuda' else False


In [70]:
# # binary segmentation (this parameters are default when you call Unet('resnet34')
# model = sm.Unet('resnet34', classes=1, activation='sigmoid')

# BACKBONE = 'resnet34'
# #preprocess_input = sm.get_preprocessing(BACKBONE)

# learning_rate = 1e-3
# weight_decay  = 1e-3
# nepochs =10
# criterion = torch.nn.BCEWithLogitsLoss()
# # optimizer_sgd = torch.optim.Adam(model.params(), lr=learning_rate, momentum=0.9,weight_decay=weight_decay)
# optimizer_sgd = torch.optim.Adam(model, lr=learning_rate, weight_decay=weight_decay)
import segmentation_models_pytorch as smp

# Define the model
model = smp.Unet('resnet34', classes=1, activation='sigmoid')

# Specify the optimizer, Set Hyperparameters
learning_rate = 0.001
weight_decay = 0.0001
nepochs =10
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#Load our data in our special dataloader
TissTrainData = OcelotDatasetLoader(paths = DATA_PATHS, dataToLoad = 'Tissue') #d_transforms


#Establish a train/validation split size
TrainValSplit = [int(0.8*len(TissTrainData)), len(TissTrainData) - int(0.8*len(TissTrainData))]

#Establish our training and validation data using torch's built in random_split on our own formatted data
TrainingData, ValidationData = torch.utils.data.random_split(TissTrainData, TrainValSplit)

batch_size = 128
dataloaderTrain = torch.utils.data.DataLoader(TrainingData, batch_size=batch_size, num_workers=4)
dataloaderVal = torch.utils.data.DataLoader(ValidationData, batch_size=batch_size,  num_workers=4)
# test_loader = torch.utils.data.DataLoader(TestingData, batch_size=batch_size,  num_workers=4)
# # preprocess input
# x_train = preprocess_input(x_train)
# x_val = preprocess_input(x_val)

# model.fit(
#    x=x_train,
#    y=y_train,
#    batch_size=16,
#    epochs=100,
#    validation_data=(x_val, y_val),
# )

def compute_accuracy_segmentation(model, data_loader):
    model.eval()
    total_correct_pixels = 0
    total_pixels = 0

    with torch.no_grad():
        for data in data_loader:
            images = data['image']
            targets = data['mask']

            images = images.to(device)
            targets = targets.to(device)

            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            correct_pixels = torch.sum(predicted_masks == targets)
            total_correct_pixels += correct_pixels.item()

            total_pixels += targets.numel()

    accuracy = total_correct_pixels / total_pixels
    return accuracy


def myTrainingLoop(model,dataloaderTrain,dataloaderVal,optimizer,nepoch,my_device,criterion):
    ## Training Loop
    
    model= model.to(dtype= dtype, device = my_device) ## Sending the model to GPU
    
    ## Setting Up some paramters for analysis
    full_loss=[]
    training_accuracy_values =[]
    validation_accuracy_values =[]
    best_val_acc=0
    
    # Until All Epochs are done
    for e in range(nepoch):
        print("Runing Epoch ",e)
        for temp,temp1,temp2,temp3,temp4 in dataloaderTrain: ## Iterator over the data Loader
            image = temp2
            target = temp3
            image = torch.unsqueeze(image,1)
            ## Put Model in Training Mode
            model.train() 
            
            ## Send inputs to GPU
            image = image.to(dtype= dtype, device = my_device) 
            target = target.to(dtype= torch.long, device = my_device)
            
            ## Forward pass
            output = model(image)
            classification_loss = criterion(output, target)
            
            ## Set Gradients to Zero
            optimizer.zero_grad()
            
            ## Backward pass
            classification_loss.backward()
            
            ## Optimizer Step
            optimizer.step()
            
            full_loss.append(classification_loss.item())
        val_accu = compute_accuracy_segmentation(model,dataloaderVal)
        train_accu = compute_accuracy_segmentation(model,dataloaderTrain)
        training_accuracy_values.append(train_accu)
        validation_accuracy_values.append(val_accu)
        print('Validation Accuracy ',val_accu.cpu().numpy())
        if(val_accu>best_val_acc):
            best_val_acc = val_accu
            best_trained_model=copy.deepcopy(model.state_dict())
            torch.save(best_trained_model,'./best_trained_model.pt')
    return training_accuracy_values, validation_accuracy_values,full_loss

#Load into Pytorch's built in DataLoader
TissTrainLoader = DataLoader(dataset = TrainingData, batch_size=batch_size, num_workers=4)
TissValLoader = DataLoader(ValidationData, batch_size=batch_size, num_workers=4)

# define model
model = smp.Unet(BACKBONE, encoder_weights='imagenet')


In [71]:
start_time = time.time()
train_acc,val_acc,full_loss = myTrainingLoop(model,TissTrainLoader,TissValLoader,optimizer,nepochs,my_device,criterion)
print("Total Runtime", time.time()-start_time)
plt.plot(full_loss,label='Training loss')